# Bi-LSTM on AGnews

## Librairies

In [ ]:
# !pip install transformers==4.8.2
# !pip install datasets==1.7.0

In [2]:
import os
import sys

In [3]:
import time
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorWithPadding

from esntorch.utils.embedding import EmbeddingModel

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Global variables

In [5]:
MODEL_FILE = '~/Results/Bi-LSTM/Bi-LSTM/AGnews-4_bilstm-128_10ep_model.pt'
RESULTS_FILE = '~/Results/Bi-LSTM/Bi-LSTM/AGnews-4_bilstm-128_10ep_results.pkl'

## Dataset

In [6]:
# rename correct column as 'labels': depends on the dataset you load

def load_and_enrich_dataset(dataset_name, split, cache_dir):
    
    dataset = load_dataset(dataset_name, split=split, cache_dir=CACHE_DIR)
    
    dataset = dataset.rename_column('label', 'labels') # 'label-fine'
    dataset = dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding=False), batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    def add_lengths(sample):
        sample["lengths"] = sum(sample["input_ids"] != 0)
        return sample
    
    dataset = dataset.map(add_lengths, batched=False)
    
    return dataset

In [ ]:
CACHE_DIR = 'cache_dir/' # put your path here

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

full_train_dataset = load_and_enrich_dataset('ag_news', split='train', cache_dir=CACHE_DIR).sort("lengths") # toriving/sst5
train_val_datasets = full_train_dataset.train_test_split(train_size=0.8, shuffle=True)
train_dataset = train_val_datasets['train'].sort("lengths")
val_dataset = train_val_datasets['test'].sort("lengths")

test_dataset = load_and_enrich_dataset('ag_news', split='test', cache_dir=CACHE_DIR).sort("lengths")

dataset_d = {
    'full_train': full_train_dataset,
    'train': train_dataset,
    'val': val_dataset,
    'test': test_dataset
    }

dataloader_d = {}
for k, v in dataset_d.items():
    dataloader_d[k] = torch.utils.data.DataLoader(v, batch_size=256, collate_fn=DataCollatorWithPadding(tokenizer))

In [8]:
dataset_d

{'full_train': Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 120000
 }),
 'train': Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 96000
 }),
 'val': Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 24000
 }),
 'test': Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 7600
 })}

## Model

In [9]:
class RNN(nn.Module):
    
    def __init__(self, embedding_dim,
                 hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        super().__init__()
        
        self.embedding = EmbeddingModel('bert-base-uncased', device)
        
        # recurrent hidden layers (LSTM)
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        # fully connected output layer (fc)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, batch):
        
        packed_embedded = self.dropout(self.embedding.get_embedding(batch))        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)      
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [10]:
output_dim = len(set(full_train_dataset['labels'].tolist()))

In [11]:
output_dim

4

In [12]:
# Parameters

# INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 768
HIDDEN_DIM = 128 # 256 was default
OUTPUT_DIM = output_dim # 1 (binary) or output_dim (*** multiclass ***)
N_LAYERS = 1 # 2 was default
BIDIRECTIONAL = True
DROPOUT = 0.5
# PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [13]:
# Model

model = RNN(EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT)

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [14]:
def count_parameters(model):
    
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 920,580 trainable parameters


## Training

In [15]:
# Adam optimizer
optimizer = optim.Adam(model.parameters())

In [16]:
# Criterion
# criterion = nn.BCEWithLogitsLoss() # binary
criterion = nn.CrossEntropyLoss() # *** multiclass ***

In [17]:
# Put model and criterion to device
model = model.to(device)
criterion = criterion.to(device)

In [18]:
# binary accuracy
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    # round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds)) # thresholding output neuron
    correct = (rounded_preds == y).float()            # convert into float for division 
    acc = correct.sum() / len(correct)
    
    return acc

In [19]:
# *** multiclass ***
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    
    return correct.sum().item() / torch.FloatTensor([y.shape[0]])

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions_raw = model(batch)
                        
        predictions = predictions_raw.squeeze(1)
                
        loss = criterion(predictions, batch['labels'])
        
        # acc = binary_accuracy(predictions, batch.label)        # *** binary *** 
        acc = categorical_accuracy(predictions, batch['labels']) # *** multiclass ***
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch).squeeze(1)
            
            loss = criterion(predictions, batch['labels'])
            
            # acc = binary_accuracy(predictions, batch.label)        # *** binary ***
            acc = categorical_accuracy(predictions, batch['labels']) # *** multiclass ***

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
import time

def epoch_time(start_time, end_time):
    
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 10 # 30

best_valid_loss = float('inf')

t0 = time.time()

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, dataloader_d['train'], optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, dataloader_d['val'], criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_FILE)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

t1 = time.time()
train_time = t1 - t0
print('training time: {}'.format(train_time))

Epoch: 01 | Epoch Time: 4m 47s
	Train Loss: 0.324 | Train Acc: 88.36%
	 Val. Loss: 0.335 |  Val. Acc: 88.67%
Epoch: 02 | Epoch Time: 4m 44s
	Train Loss: 0.238 | Train Acc: 91.64%
	 Val. Loss: 0.248 |  Val. Acc: 91.45%
Epoch: 03 | Epoch Time: 4m 45s
	Train Loss: 0.215 | Train Acc: 92.31%
	 Val. Loss: 0.396 |  Val. Acc: 86.37%
Epoch: 04 | Epoch Time: 4m 45s
	Train Loss: 0.202 | Train Acc: 92.83%
	 Val. Loss: 0.237 |  Val. Acc: 92.06%
Epoch: 05 | Epoch Time: 4m 46s
	Train Loss: 0.186 | Train Acc: 93.45%
	 Val. Loss: 0.223 |  Val. Acc: 92.56%
Epoch: 06 | Epoch Time: 4m 44s
	Train Loss: 0.176 | Train Acc: 93.71%
	 Val. Loss: 0.224 |  Val. Acc: 92.56%
Epoch: 07 | Epoch Time: 4m 45s
	Train Loss: 0.165 | Train Acc: 94.13%
	 Val. Loss: 0.220 |  Val. Acc: 92.87%
Epoch: 08 | Epoch Time: 4m 45s
	Train Loss: 0.156 | Train Acc: 94.44%
	 Val. Loss: 0.225 |  Val. Acc: 92.84%
Epoch: 09 | Epoch Time: 4m 45s
	Train Loss: 0.147 | Train Acc: 94.77%
	 Val. Loss: 0.211 |  Val. Acc: 93.09%
Epoch: 10 | Epoch T

In [24]:
model = RNN(EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT)
model.load_state_dict(torch.load(MODEL_FILE))
model.eval().to(device)

RNN(
  (rnn): LSTM(768, 128, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=256, out_features=4, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [25]:
test_loss, test_acc = evaluate(model, dataloader_d['test'], criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

results = test_acc, train_time, count_parameters(model)

Test Loss: 0.220 | Test Acc: 93.18%


In [26]:
results

(0.9318181832631429, 2855.069205760956, 920580)

In [27]:
with open(RESULTS_FILE, 'wb') as fh:
    pickle.dump(results, fh)